# Distributed dialogue

This example initializes an assistant agent and a user agent as separate processes and uses RPC to communicate between them. The full codes can be found in in `examples/distributed/distributed_dialog.py`

To install AgentScope, please follow the steps in [README.md](https://github.com/alibaba/AgentScope/blob/main/README.md#installation).

In this case, we need to initialize two agents: assistant agent and user agnent.

The code for assistant is as follows.

```bash
def setup_assistant_server(assistant_host: str, assistant_port: int) -> None:
    """Set up assistant rpc server"""
    agentscope.init(
        model_configs="configs/model_configs.json",
    )
    assistant_server_launcher = RpcAgentServerLauncher(
        name="Assitant",
        agent_class=RpcDialogAgent,
        host=assistant_host,
        port=assistant_port,
        sys_prompt="You are a helpful assistant.",
        model="gpt-3.5-turbo",
        use_memory=True,
        local_mode=False,
    )
    assistant_server_launcher.launch()
    assistant_server_launcher.wait_until_terminate()
```

To run two processes on jupyter at the same time, we put `assistant agent` in the background.

In [ ]:
%cd ../examples/distributed/

In [2]:
import subprocess

# Using Popen to initialize a backstage process

proc = subprocess.Popen(
    ['python', 'distributed_dialog.py', '--role', 'assistant', '--assistant-host', 'localhost', '--assistant-port', '12010'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# This code will return immediately and the process will run in the background


Then we run the user agent on jupyter.

In [ ]:
%cd ../../notebook/

Define the model_configs.

In [4]:
model_configs = [
    {
        "type": "openai",
        "name": "gpt-3.5-turbo",
        "parameters": {
            "api_key": "xxx",
            "organization_id": "xxx",
            "temperature": 0.0
        }
    },
    {
        "type": "openai",
        "name": "gpt-4",
        "parameters": {
            "api_key": "xxx",
            "organization_id": "xxx",
            "temperature": 0.0
        }
    }
]

Run the following codes to initialize the user agent.

In [ ]:
import time

import agentscope
from agentscope.agents import UserAgent
from agentscope.agents.rpc_dialog_agent import RpcDialogAgent
from agentscope.utils.logging_utils import logger

assistant_host = "localhost"
assistant_port = 12010

agentscope.init(
    model_configs=model_configs,
)
assistant_agent = RpcDialogAgent(
    name="Assistant",
    host=assistant_host,
    port=assistant_port,
    launch_server=False,
)
user_agent = UserAgent(
    name="User",
    require_url=False,
)
logger.info(
    "Setup successfully, have fun chatting! (enter 'exit' to close the "
    "agent)",
)
msg = user_agent()
while not msg.content.endswith("exit"):
    msg = assistant_agent(msg)
    logger.chat(msg.update_value())
    time.sleep(0.5)
    msg = user_agent(msg)

Finally, terminate the assistant agent in the backend. 

In [6]:
proc.terminate()